### Define dataset pre-processing functions

In [1]:
import numpy as np
import pandas as pd

# Pre-process data
import scipy.stats as stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Preprocess features
from sklearn.preprocessing import LabelEncoder

types_of_cells = 'All'
mol_profiles = 'BRCAmutmet','HRP'
therapies = 'All'

features = ['GlobalCellType','cycif.slide','TIM3','pSTAT1','CD45RO','CD20','CD11c','CD207','GranzymeB','CD163','CD4','CD3d','CD8a','FOXP3','PD1','CD15','PDL1_488','Ki67','Vimentin','MHCII','MHCI','ECadherin','aSMA','CD31','pTBK1','CK7','yH2AX','cPARP1',
            'Area','Eccentricity','Roundness','CD11c.MY','CD15.MY','CD163.MP','CD207.MY','CD31.stromal','CD4.T.cells','CD68.MP','CD8.T.cells','Cancer','Other','Other.MY','Stromal','T.regs','B.cells','Other.immune',
            'Molecular.profile2','therapy_sequence','patient']

transformation = 'LOG2' 
features_to_transform = ['TIM3','pSTAT1','CD45RO','CD20','CD11c','CD207','GranzymeB','CD163','CD4','CD3d','CD8a','FOXP3','PD1','CD15','PDL1_488','Ki67','Vimentin','MHCII','MHCI','ECadherin','aSMA','CD31','pTBK1','CK7','yH2AX','cPARP1']

operation = 'remove'
features_for_outliers = features_to_scale = ['TIM3','pSTAT1','CD45RO','CD20','CD11c','CD207','GranzymeB','CD163','CD4','CD3d','CD8a','FOXP3','PD1','CD15','PDL1_488','Ki67','Vimentin','MHCII','MHCI','ECadherin','aSMA','CD31','pTBK1','CK7','yH2AX','cPARP1',
            'Area','Eccentricity','Roundness']
        
# Scale data
scaler_dict = {'MinMax': MinMaxScaler(),'Standard': StandardScaler()}
# MinMaxScaler scales the data to be within a specific range, usually between 0 and 1
# StandardScaler scales the data to have a mean of zero and a standard deviation of one
scaler_type = 'Standard' # Scaling type
scale_by = 'slide' # Define how to scale. Available options: whole, patient, slide

# Variables for machine learning step
categorical_variables = ['Molecular.profile2', 'therapy_sequence']

def choose_features(df):
        df = df.loc[:,features]
        
        return df
    
def transform_df(df):

    try:
        if transformation == 'LOG':
            df.loc[:, features_to_transform] = np.log(df.loc[:, features_to_transform] + 1)

        elif transformation == 'LOG2':
            df.loc[:, features_to_transform] = np.log2(df.loc[:, features_to_transform] + 1)
            
        elif transformation == 'BOXCOX':
            # transform data & save lambda value
            for feature in features_to_transform:
                df[feature], _ = stats.boxcox(df[feature].values)

        else:
            raise ValueError("Invalid transformation specified.")
        
        return df
        
    except Exception as e:
        print(f"\nAn error occurred data transformation: {e}", exc_info=True)

    
            
def remove_outliers(df):
    
    try:
        if operation == 'trim_by_slide':
            slides = df['cycif.slide'].unique()
        
            for slide in slides:
                # Create a mask to filter data belonging to the current slide
                slide_mask = df['cycif.slide'] == slide
                
                for feature in features_for_outliers:
                    percentiles = np.percentile(df.loc[slide_mask, feature], [1, 99])
                    
                    # Replace values below the 1st percentile with the 1st percentile value
                    df.loc[slide_mask & (df[feature] < percentiles[0]), feature] = percentiles[0]
                    # Replace values above the 99th percentile with the 99th percentile value
                    df.loc[slide_mask & (df[feature] > percentiles[1]), feature] = percentiles[1]

        elif operation == 'remove':

            df_sub = df.loc[:, features_for_outliers]

            # Identify outliers using the 1st (0.01) and 99th (0.99) percentiles for each feature
            # For each data point, 'lim' will be True if the value is within the range [0.01, 0.99], otherwise False
            lim = np.logical_and(df_sub < df_sub.quantile(0.99, numeric_only=False),
                            df_sub > df_sub.quantile(0.01, numeric_only=False))

            # Data points outside the range [0.01, 0.99] will be replaced with NaN
            df.loc[:, features_for_outliers] = df_sub.where(lim, np.nan)
            
            # Drop rows with NaN in numerical columns
            df.dropna(subset=features_for_outliers, inplace=True)
                    
        else: 
            raise ValueError("Invalid operation specified.")
        
        return df
    
    except Exception as e:
        print(f"\nAn error occurred  outliers removal: {e}", exc_info=True)
    
def scaler(df):
    
        # Get a scaler from the dictionary of supported scaler types
        scaler = scaler_dict.get(scaler_type)

        return scaler.fit_transform(df)

def scaling(df):
    
    try:
        if scaler_type in scaler_dict:
            
            if scale_by not in ['whole','patient', 'slide']:
                raise ValueError(f"Invalid order: {scale_by}")
            
            if scale_by == 'patient':
                patients = df['patient'].unique()
                # Iterate through each unique patient ID and scale the specified features for each patient separately
                for patient in patients:
                    df.loc[df['patient'] == patient, features_to_scale] = scaler(df.loc[df['patient'] == patient, features_to_scale])
            
            elif scale_by == 'slide':
                slides = df['cycif.slide'].unique()
                # Iterate through each unique slide and scale the specified features for each slide separately
                for slide in slides:
                    df.loc[df['cycif.slide'] == slide, features_to_scale] = scaler(df.loc[df['cycif.slide'] == slide, features_to_scale])
            
            elif scale_by == 'whole':
                # Scale the specified features on the entire DataFrame
                df.loc[features_to_scale] = scaler(df.loc[features_to_scale])
            
            return df
    
    except Exception as e:
        print(f"\nAn error occurred  scaling: {e}", exc_info=True)

def choose_types_of_cells(df):
        
    # Remove Others and group immune and stromal cells
    df = df[~df["GlobalCellType"].isin(['Others','Other'])]
    df["GlobalCellType"] = df["GlobalCellType"].replace(to_replace = ['CD8.T.cells', 'B.cells', 'T.regs', 'CD4.T.cells'], value = 'Lymphocytes')
    df["GlobalCellType"] = df["GlobalCellType"].replace(to_replace = ['CD11c.MY', 'Other.MY', 'CD163.MP', 'CD207.MY', 'CD68.MP', 'CD15.MY'], value = 'Myeloids')
    df["GlobalCellType"] = df["GlobalCellType"].replace(to_replace = ['CD31.stromal'], value = 'Stromal')
    
    try:
        # Leave only chosen cell types in df
        if types_of_cells == "Non-cancer":
            df = df[~df["GlobalCellType"].isin(['Cancer'])]
            
        elif types_of_cells == "Lymphocytes-stromal":
            df = df[df["GlobalCellType"].isin(['Lymphocytes', 'Stromal'])]
            
        elif types_of_cells == "Myeloid-stromal":
            df = df[df["GlobalCellType"].isin(['Myeloids', 'Stromal'])]
            
        elif types_of_cells == "Immune":
            df = df[df["GlobalCellType"].isin(['Myeloids', 'Lymphocytes', 'Other.immune'])]

        elif types_of_cells in ["Cancer", "Stromal", "Myeloids"]:
            df = df[df["GlobalCellType"] == types_of_cells]
                
        elif types_of_cells not in ["All"]:
            raise ValueError("Invalid cell type(s) specified.")
        
    except Exception as e:
        print(f"\nAn error occurred while choosing cell type: {e}", exc_info=True)
    
    print(f"\nChose successfully cell types: {df['GlobalCellType'].unique()}.")
    
    df["GlobalCellType"] = df["GlobalCellType"].replace(to_replace = ['Myeloids', 'Lymphocytes', 'Other.immune'], value = 'Immune')

    cell_types = df["GlobalCellType"].unique()

    unique_patients = df['patient'].unique()
    
    # Remove patients with too few cells for each chosen cell type
    for cell_type in cell_types:
        # Group df by patient and get the size of each group
        grouped_df = df[df['GlobalCellType'] == cell_type].groupby('patient').size()

        # Filter patients with 100 or fewer cells and store them
        selected_patients = grouped_df[grouped_df <= 100].index

        # Remove patients with 100 and fewer cells
        if len(selected_patients) > 0:
            df = df[~df['patient'].isin(selected_patients)]

    unique_patients = df['patient'].unique()
    print(f"\nAfter thresholding number of patients: {len(unique_patients)}")
    
    # Remove column GlobalCellType
    df = df.drop(columns='GlobalCellType')

    return df
        
def choose_mol_profiles(df):
    
    if mol_profiles != 'All':
        df = df[df['Molecular.profile2'].isin(mol_profiles)]
    else:
        df = df[df['Molecular.profile2'].isin(['BRCAmutmet','HRD','HRP','CCNE1amp'])]
        
    return df

def choose_therapy_sequences(df):
    if therapies != 'All':
        df = df[df['therapy_sequence'] == therapies]
    
    return df

def prepare_categorical_inputs(df):

    # Encode categorical variables 
    label_encoder_dict = {}
    for variable in categorical_variables:
        
        label_encoder = LabelEncoder()
        
        # Convert the values of the current categorical variable to strings and encode them
        # The encoded values will replace the original values in the DataFrame
        df[variable] = label_encoder.fit_transform(df[variable].astype('str'))
        
        # Save the encoding results for the current variable in 'label_encoder_dict'
        # The dictionary will store the unique class labels and their corresponding encoded values
        label_encoder_dict[variable] = {
            'label_codes': label_encoder.classes_.tolist(), 
            'label_values': label_encoder.transform(label_encoder.classes_).tolist()
        }
        
        print(label_encoder_dict)
        
    
    return df

def prep_train_test_data(df):

    # To receive same number of patients independent of runs
    np.random.seed(33)
    
    # Group df by Molecular profile and therapy
    grouped = df.groupby(['Molecular.profile2', 'therapy_sequence'])

    # Assign 80% of patients from each unique group of Molecular profile and therapy to training and rest to test sets
    X_train_full = grouped.apply(lambda x: x.loc[x['patient'].isin(np.random.choice(x['patient'].unique(), size=int(0.8*len(x['patient'].unique())), replace=False))])
    X_test_full = grouped.apply(lambda x: x.loc[x['patient'].isin(np.setdiff1d(x['patient'].unique(), X_train_full['patient']))])
    
    X_train_full.reset_index(drop=True, inplace=True)
    X_test_full.reset_index(drop=True, inplace=True)
    
    # Assign target columns to y_train and y_test
    y_train=X_train_full['Molecular.profile2']
    y_test=X_test_full['Molecular.profile2']
    
    # Drop target columns
    X_train = X_train_full.drop(columns = ['Molecular.profile2', 'patient', 'cycif.slide'])
    X_test = X_test_full.drop(columns = ['Molecular.profile2', 'patient', 'cycif.slide'])
    
    return X_train_full.drop(columns = ['patient', 'cycif.slide']),X_test_full.drop(columns = ['patient', 'cycif.slide']),X_train,X_test,y_train,y_test

### Load dataset

In [2]:
df = pd.read_csv("final_dataset_202403.csv")
df['Molecular.profile2'] = df['Molecular.profile2'].replace('BRCAmut/met', 'BRCAmutmet')

### Characterise dataset

In [3]:
print(df['Molecular.profile2'].unique())
print(df['therapy_sequence'].unique())
print(len(df['patient'].unique()))

['Other' 'BRCAmutmet' 'HRD' 'HRP' 'CCNE1amp']
['PDS' 'IDS']
233


### Pre-process dataset

In [4]:
df = choose_features(df)
print(len(df['patient'].unique()))
df = transform_df(df)
print(len(df['patient'].unique()))
df = remove_outliers(df)
print(len(df['patient'].unique()))
df = scaling(df)
print(len(df['patient'].unique()))
df = choose_types_of_cells(df)
print(len(df['patient'].unique()))
df = choose_mol_profiles(df)
print(len(df['patient'].unique()))
df = choose_therapy_sequences(df)
print(len(df['patient'].unique()))

df = prepare_categorical_inputs(df)
print(len(df['patient'].unique()))
X_train_full,X_test_full,X_train,X_test,y_train,y_test = prep_train_test_data(df)

233
233
233
233


/tmp/ipykernel_492123/2367332574.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["GlobalCellType"] = df["GlobalCellType"].replace(to_replace = ['CD8.T.cells', 'B.cells', 'T.regs', 'CD4.T.cells'], value = 'Lymphocytes')
/tmp/ipykernel_492123/2367332574.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["GlobalCellType"] = df["GlobalCellType"].replace(to_replace = ['CD11c.MY', 'Other.MY', 'CD163.MP', 'CD207.MY', 'CD68.MP', 'CD15.MY'], value = 'Myeloids')
/tmp/ipykernel_492123/2367332574.py:148:


Chose successfully cell types: ['Cancer' 'Stromal' 'Lymphocytes' 'Myeloids' 'Other.immune'].


/tmp/ipykernel_492123/2367332574.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["GlobalCellType"] = df["GlobalCellType"].replace(to_replace = ['Myeloids', 'Lymphocytes', 'Other.immune'], value = 'Immune')



After thresholding number of patients: 224
224
105
105
{'Molecular.profile2': {'label_codes': ['BRCAmutmet', 'HRP'], 'label_values': [0, 1]}}
{'Molecular.profile2': {'label_codes': ['BRCAmutmet', 'HRP'], 'label_values': [0, 1]}, 'therapy_sequence': {'label_codes': ['IDS', 'PDS'], 'label_values': [0, 1]}}
105


### Set up ML experiment

In [5]:
import mlflow 
mlflow.set_tracking_uri("http://127.0.0.1:8080")

In [6]:
full_df = pd.concat([X_train_full, X_test_full], ignore_index=True)
print(full_df.shape)

(1329491, 46)


In [7]:
print(X_train_full.shape)
print(X_test_full.shape)

(1036939, 46)
(292552, 46)


In [8]:
import pandas as pd
from pycaret.classification import *

# Init setup
clf1 = setup(X_train_full, target='Molecular.profile2', log_experiment = 'mlflow', experiment_name = 'Two_classes_202403', test_data=X_test_full, index = False, preprocess=False)

,Description,Value
0,Session id,6519
1,Target,Molecular.profile2
2,Target type,Binary
3,Original data shape,"(1329491, 46)"
4,Transformed data shape,"(1329491, 46)"
5,Transformed train set shape,"(1036939, 46)"
6,Transformed test set shape,"(292552, 46)"
7,Numeric features,45


2024/05/02 14:43:34 INFO mlflow.tracking.fluent: Experiment with name 'Two_classes_202403' does not exist. Creating a new experiment.


### Two classes predictions

In [9]:
rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6746,0.6727,0.3378,0.5407,0.4158,0.2066,0.2176
1,0.7552,0.8451,0.7858,0.6112,0.6876,0.4915,0.5020
2,0.7285,0.6540,0.4308,0.6592,0.5211,0.3430,0.3581
3,0.6843,0.7578,0.6973,0.5302,0.6024,0.3486,0.3576
4,0.7303,0.7771,0.6474,0.5987,0.6221,0.4130,0.4137
5,0.7731,0.8609,0.3796,0.9019,0.5343,0.4156,0.4836
6,0.8658,0.9226,0.7484,0.8426,0.7927,0.6940,0.6967
7,0.7041,0.8245,0.2198,0.7266,0.3375,0.2120,0.2750
8,0.6000,0.3645,0.0521,0.1926,0.0821,-0.0750,-0.1013


In [10]:
tuned_rf = tune_model(rf)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6571,0.5995,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.6571,0.6660,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.6571,0.6814,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.6571,0.6873,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.6571,0.6877,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.6571,0.7275,0.0000,0.0000,0.0000,0.0000,0.0000
6,0.6571,0.8312,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.6571,0.7824,0.0000,0.0000,0.0000,0.0000,0.0000
8,0.6571,0.5436,0.0000,0.0000,0.0000,0.0000,0.0000


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [11]:
evaluate_model(tuned_rf)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [12]:
final_model = finalize_model(tuned_rf)

In [13]:
predict_model(final_model, data = X_test, raw_score=True)

,TIM3,pSTAT1,CD45RO,CD20,CD11c,CD207,GranzymeB,CD163,CD4,CD3d,...,Other,Other.MY,Stromal,T.regs,B.cells,Other.immune,therapy_sequence,prediction_label,prediction_score_0,prediction_score_1
0,-1.761724,-1.792585,-1.892670,-0.899680,-0.957330,-1.109026,-0.638731,-1.382635,-1.726475,0.076979,...,0.0,0.000000,0.866667,0.0,0.0,0.0,0,0,0.98,0.02
1,-1.252074,-1.491846,-1.282688,-0.983397,-0.632799,-0.609742,-0.127029,-1.119683,-1.489874,-0.457687,...,0.0,0.000000,1.000000,0.0,0.0,0.0,0,0,0.91,0.09
2,-1.513838,-1.514140,-1.567259,-1.044543,-0.859811,-0.782034,-0.059064,-1.254908,-1.776814,-0.703703,...,0.0,0.000000,0.923077,0.0,0.0,0.0,0,0,0.90,0.10
3,-1.494061,-1.609426,-1.412769,-0.837891,-0.926374,-0.387337,0.357668,-1.107483,-1.549405,0.110740,...,0.0,0.000000,0.875000,0.0,0.0,0.0,0,0,0.94,0.06
4,-1.727331,-1.605869,-1.752560,-1.000690,-0.840959,-1.182580,-0.330135,-1.323476,-1.709097,-0.654664,...,0.0,0.000000,1.000000,0.0,0.0,0.0,0,0,0.92,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292547,-0.508372,-1.285522,-1.084351,-0.228052,2.062762,3.049381,1.358564,1.452926,0.363234,-0.317026,...,0.0,0.000000,0.000000,0.0,0.0,0.0,1,1,0.18,0.82
292548,1.564123,0.962397,1.788038,-0.233907,0.683153,2.176982,3.352562,0.997400,1.660157,0.425261,...,0.0,0.000000,0.166667,0.0,0.0,0.0,1,1,0.24,0.76
292549,1.138016,0.901402,1.358302,-0.327525,0.487023,0.587608,2.842589,0.716505,1.082947,0.181159,...,0.0,0.000000,0.166667,0.0,0.0,0.0,1,1,0.23,0.77
292550,-0.863623,-0.682099,-1.473688,-0.170834,1.905892,0.857797,1.594102,0.984592,-0.422437,-0.350603,...,0.0,0.000000,0.000000,0.0,0.0,0.0,1,1,0.23,0.77


In [14]:
predict_model(final_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


,TIM3,pSTAT1,CD45RO,CD20,CD11c,CD207,GranzymeB,CD163,CD4,CD3d,...,Other,Other.MY,Stromal,T.regs,B.cells,Other.immune,therapy_sequence,Molecular.profile2,prediction_label,prediction_score
1036939,-1.761724,-1.792585,-1.892670,-0.899680,-0.957330,-1.109026,-0.638731,-1.382635,-1.726475,0.076979,...,0.0,0.000000,0.866667,0.0,0.0,0.0,0,0,0,0.98
1036940,-1.252074,-1.491846,-1.282688,-0.983397,-0.632799,-0.609742,-0.127029,-1.119683,-1.489874,-0.457687,...,0.0,0.000000,1.000000,0.0,0.0,0.0,0,0,0,0.91
1036941,-1.513838,-1.514140,-1.567259,-1.044543,-0.859811,-0.782034,-0.059064,-1.254908,-1.776814,-0.703703,...,0.0,0.000000,0.923077,0.0,0.0,0.0,0,0,0,0.90
1036942,-1.494061,-1.609426,-1.412769,-0.837891,-0.926374,-0.387337,0.357668,-1.107483,-1.549405,0.110740,...,0.0,0.000000,0.875000,0.0,0.0,0.0,0,0,0,0.94
1036943,-1.727331,-1.605869,-1.752560,-1.000690,-0.840959,-1.182580,-0.330135,-1.323476,-1.709097,-0.654664,...,0.0,0.000000,1.000000,0.0,0.0,0.0,0,0,0,0.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329486,-0.508372,-1.285522,-1.084351,-0.228052,2.062762,3.049381,1.358564,1.452926,0.363234,-0.317026,...,0.0,0.000000,0.000000,0.0,0.0,0.0,1,1,1,0.82
1329487,1.564123,0.962397,1.788038,-0.233907,0.683153,2.176982,3.352562,0.997400,1.660157,0.425261,...,0.0,0.000000,0.166667,0.0,0.0,0.0,1,1,1,0.76
1329488,1.138016,0.901402,1.358302,-0.327525,0.487023,0.587608,2.842589,0.716505,1.082947,0.181159,...,0.0,0.000000,0.166667,0.0,0.0,0.0,1,1,1,0.77
1329489,-0.863623,-0.682099,-1.473688,-0.170834,1.905892,0.857797,1.594102,0.984592,-0.422437,-0.350603,...,0.0,0.000000,0.000000,0.0,0.0,0.0,1,1,1,0.77


In [15]:
save_model(final_model, 'Two_classes_model_202403')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('clean_column_names',
                  TransformerWrapper(exclude=None, include=None,
                                     transformer=CleanColumnNames(match='[\\]\\[\\,\\{\\}\\"\\:]+'))),
                 ('actual_estimator',
                  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                         class_weight=None, criterion='gini',
                                         max_depth=None, max_features='sqrt',
                                         max_leaf_nodes=None, max_samples=None,
                                         min_impurity_decrease=0.0,
                                         min_samples_leaf=1, min_samples_split=2,
                                         min_weight_fraction_leaf=0.0,
                                         monotonic_cst=None, n_estimators=100,
                                         n_jobs=-1, oob_score=False,
                                     